In [ ]:
!pip install indic-doctr

  Using cached pypdfium2-3.21.1-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.8 MB)
  Attempting uninstall: pypdfium2
    Found existing installation: pypdfium2 4.30.0
    Uninstalling pypdfium2-4.30.0:
      Successfully uninstalled pypdfium2-4.30.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python-doctr 0.9.0a0 requires pypdfium2<5.0.0,>=4.0.0, but you have pypdfium2 3.21.1 which is incompatible.


In [ ]:
# Colab related installations to install pyproject.toml projects correctly
!sudo apt install libcairo2-dev pkg-config
!pip3 install pycairo
# Install the most up-to-date version from GitHub
# TensorFlow
# !pip install python-doctr[tf]@git+https://github.com/mindee/doctr.git
# PyTorch
!pip3 install python-doctr[torch]@git+https://github.com/mindee/doctr.git
# Restart runtime
exit()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcairo2-dev is already the newest version (1.16.0-5ubuntu2).
pkg-config is already the newest version (0.29.2-1ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
  Cloning https://github.com/mindee/doctr.git to /tmp/pip-install-fs6qcibh/python-doctr_04c4ae3ae82c4134a635d9c78f0ae1b2
  Running command git clone --filter=blob:none --quiet https://github.com/mindee/doctr.git /tmp/pip-install-fs6qcibh/python-doctr_04c4ae3ae82c4134a635d9c78f0ae1b2
  Resolved https://github.com/mindee/doctr.git to commit b9c4e6d9328a224e43abf3d37556f6337e22d755
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install python-doctr[torch]

  Using cached pypdfium2-4.30.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.8 MB)
  Attempting uninstall: pypdfium2
    Found existing installation: pypdfium2 3.21.1
    Uninstalling pypdfium2-3.21.1:
      Successfully uninstalled pypdfium2-3.21.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
indic-doctr 0.7.1a0 requires pypdfium2<4.0.0,>=3.3.0, but you have pypdfium2 4.30.0 which is incompatible.


In [ ]:
!pip install tf2onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 6.2 MB/s eta 0:00:00


In [ ]:
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

predictor = ocr_predictor(det_arch='db_resnet50', reco_arch='crnn_vgg16_bn', pretrained=True)
doc = DocumentFile.from_images(r"a.jpg")
result = predictor(doc)

json_export = result.export()
d = json_export
print(json_export)

  0%|          | 0/94122142 [00:00<?, ?it/s]

{'pages': [{'page_idx': 0, 'dimensions': (576, 1280), 'orientation': {'value': None, 'confidence': None}, 'language': {'value': None, 'confidence': None}, 'blocks': [{'geometry': ((0.5126953125, 0.11588541666666669), (0.5302734375, 0.1332465277777778)), 'objectness_score': 0.32074037194252014, 'lines': [{'geometry': ((0.5126953125, 0.11588541666666669), (0.5302734375, 0.1332465277777778)), 'objectness_score': 0.32074037194252014, 'words': [{'value': 'A', 'confidence': 0.7796182036399841, 'geometry': ((0.5126953125, 0.11588541666666669), (0.5302734375, 0.1332465277777778)), 'objectness_score': 0.32074037194252014, 'crop_orientation': {'value': 0, 'confidence': None}}]}], 'artefacts': []}, {'geometry': ((0.6884765625, 0.265625), (0.7724609375, 0.3046875)), 'objectness_score': 0.41798774401346844, 'lines': [{'geometry': ((0.6884765625, 0.265625), (0.7724609375, 0.3046875)), 'objectness_score': 0.41798774401346844, 'words': [{'value': 'SAP', 'confidence': 0.9961374402046204, 'geometry': ((

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import json

image_path = "/content/a.jpg"
image = cv2.imread(image_path)
orig_height, orig_width = image.shape[:2]

# Get the dimensions used by the OCR model from the first page
ocr_width, ocr_height = json_export['pages'][0]['dimensions']

# Calculate the scaling factors
width_ratio = orig_width / ocr_width
height_ratio = orig_height / ocr_height

# Convert relative bounding box to absolute bounding box in xyxy format, adjusted by the scaling factors
def relative_to_absolute_bbox(rel_bbox, width_ratio, height_ratio):
    x_min_rel, y_min_rel = rel_bbox[0]
    x_max_rel, y_max_rel = rel_bbox[1]
    x_min = int(x_min_rel * ocr_width * width_ratio)
    y_min = int(y_min_rel * ocr_height * height_ratio)
    x_max = int(x_max_rel * ocr_width * width_ratio)
    y_max = int(y_max_rel * ocr_height * height_ratio)
    return [x_min, y_min, x_max, y_max]

# Collect words and their bounding boxes
word_boxes = []
for page in json_export['pages']:
    for block in page['blocks']:
        for line in block['lines']:
            for word in line['words']:
                abs_bbox = relative_to_absolute_bbox(word['geometry'], width_ratio, height_ratio)
                word_boxes.append({"word": word['value'], "bounding_box": abs_bbox})

# Desired JSON format
output_json = word_boxes

# Write to JSON file
with open("output.json", "w") as f:
    json.dump(output_json, f, indent=4)

print(json.dumps(output_json, indent=4))

[
    {
        "word": "A",
        "bounding_box": [
            656,
            66,
            678,
            76
        ]
    },
    {
        "word": "SAP",
        "bounding_box": [
            881,
            153,
            908,
            169
        ]
    },
    {
        "word": "CODE",
        "bounding_box": [
            905,
            154,
            938,
            170
        ]
    },
    {
        "word": "IMFCPO3",
        "bounding_box": [
            942,
            156,
            988,
            175
        ]
    },
    {
        "word": "suae",
        "bounding_box": [
            488,
            105,
            770,
            190
        ]
    },
    {
        "word": "919",
        "bounding_box": [
            525,
            178,
            595,
            224
        ]
    },
    {
        "word": "goe-3ge",
        "bounding_box": [
            591,
            185,
            726,
            225
        ]
    },
    {
        "word

In [ ]:
import cv2
import matplotlib.pyplot as plt

def draw_bounding_boxes(image_path, json_path):
    # Load the image
    image = cv2.imread(image_path)

    # Load the JSON file with bounding boxes
    with open(json_path, 'r') as f:
        bounding_boxes = json.load(f)

    # Draw bounding boxes and texts on the image
    for item in bounding_boxes:
        word = item['word']
        bbox = item['bounding_box']
        x_min, y_min, x_max, y_max = bbox

        # Draw the bounding box
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

        # Put the text
        cv2.putText(image, word, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0, 0), 2)

    # Display the image with bounding boxes
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Path to the image and the output JSON file
image_path = "/content/a.jpg"
json_path = "output.json"

# Call the function to draw bounding boxes
draw_bounding_boxes(image_path, json_path)

In [16]:
!pip install surya_ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.3/94.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 64.2 MB/s eta 0:00:00
  Using cached pypdfium2-4.30.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.8 MB)
  Attempting uninstall: pypdfium2
    Found existing installation: pypdfium2 3.21.1
    Uninstalling pypdfium2-3.21.1:
      Successfully uninstalled pypdfium2-3.21.1
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.3.0
    Uninstalling Pillow-9.3.0:
      Successfully uninstalled Pillow-9.3.0
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.6.0.66
    Uninstalling opencv-python-4.6.0.66:
      Successfully uninstalled opencv-python-4.6.0.66
ERROR: pip's dependency resolver does not currently take into account all th

In [ ]:
from PIL import Image
from surya.ocr import run_ocr
from surya.model.detection import segformer
from surya.model.recognition.model import load_model
from surya.model.recognition.processor import load_processor

image = Image.open('a.jpg')
langs = ["hi"] # Replace with your languages
det_processor, det_model = segformer.load_processor(), segformer.load_model()
rec_model, rec_processor = load_model(), load_processor()

predictions = run_ocr([image], [langs], det_model, det_processor, rec_model, rec_processor)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loaded recognition model vikp/surya_rec on device cpu with dtype torch.float32


Recognizing Text: 100%|██████████| 1/1 [00:59<00:00, 59.11s/it]


In [5]:
predictions

[OCRResult(text_lines=[TextLine(polygon=[[611.0, 62.0], [686.0, 65.0], [683.0, 79.0], [608.0, 77.0]], confidence=nan, text='', bbox=[611.0, 62.0, 686.0, 79.0]), TextLine(polygon=[[879.0, 152.0], [992.0, 159.0], [988.0, 175.0], [874.0, 168.0]], confidence=0.9703773260116577, text='| SAP CODE : IMFCP03 |', bbox=[879.0, 152.0, 992.0, 175.0]), TextLine(polygon=[[594.0, 197.0], [701.0, 197.0], [701.0, 210.0], [594.0, 210.0]], confidence=0.4589460790157318, text='दल- सामा', bbox=[594.0, 197.0, 701.0, 210.0]), TextLine(polygon=[[879.0, 207.0], [973.0, 207.0], [967.0, 387.0], [872.0, 387.0]], confidence=nan, text='', bbox=[879.0, 207.0, 973.0, 387.0]), TextLine(polygon=[[181.0, 371.0], [210.0, 371.0], [210.0, 382.0], [181.0, 382.0]], confidence=nan, text='', bbox=[181.0, 371.0, 210.0, 382.0]), TextLine(polygon=[[499.0, 390.0], [717.0, 398.0], [713.0, 421.0], [494.0, 413.0]], confidence=0.9902210831642151, text='24 Pieces X 500 g Pouch', bbox=[499.0, 390.0, 717.0, 421.0]), TextLine(polygon=[[57

In [16]:
json_export = predictions[0].json()
d = json_export
print(json_export)

{"text_lines":[{"polygon":[[611.0,62.0],[686.0,65.0],[683.0,79.0],[608.0,77.0]],"confidence":null,"text":"","bbox":[611.0,62.0,686.0,79.0]},{"polygon":[[879.0,152.0],[992.0,159.0],[988.0,175.0],[874.0,168.0]],"confidence":0.9703773260116577,"text":"| SAP CODE : IMFCP03 |","bbox":[879.0,152.0,992.0,175.0]},{"polygon":[[594.0,197.0],[701.0,197.0],[701.0,210.0],[594.0,210.0]],"confidence":0.4589460790157318,"text":"दल- सामा","bbox":[594.0,197.0,701.0,210.0]},{"polygon":[[879.0,207.0],[973.0,207.0],[967.0,387.0],[872.0,387.0]],"confidence":null,"text":"","bbox":[879.0,207.0,973.0,387.0]},{"polygon":[[181.0,371.0],[210.0,371.0],[210.0,382.0],[181.0,382.0]],"confidence":null,"text":"","bbox":[181.0,371.0,210.0,382.0]},{"polygon":[[499.0,390.0],[717.0,398.0],[713.0,421.0],[494.0,413.0]],"confidence":0.9902210831642151,"text":"24 Pieces X 500 g Pouch","bbox":[499.0,390.0,717.0,421.0]},{"polygon":[[579.0,468.0],[788.0,474.0],[785.0,494.0],[577.0,488.0]],"confidence":0.6089131236076355,"text":"म

In [17]:
d.keys()

AttributeError: 'str' object has no attribute 'keys'